# Python
## pandas

1. Series


- **整数索引问题**：Series 对象默认是按照标签索引.a的index其实也被截取了，那么如果对b索引，如果b[2]，其实取出来就是2，因为默认10是标签。采用*loc/iloc*方法即可对下标和标签进行区分。b.loc[2]，这里强制认为10是标签；b.iloc[0]，这里强制认为10是下标

In [35]:
from builtins import print
import pandas as pd
import numpy as np
a=pd.Series(np.arange(4))
b=a[2:,]
print(a)
print(b)
print(b.loc[2]) #把2认为是标签，默认也是这么认为的
print(b.iloc[0]) #把0认为是对应的下标

0    0
1    1
2    2
3    3
dtype: int32
2    2
3    3
dtype: int32
a 2
2


- **数据对齐**：pandas在进行两个series对象运算时，会按照index对齐后进行计算.在numpy中，形状不一致的array是不能直接相加的，在pandas中不同长度的series对象可以相加，只不过多余的值会被标记为缺失值。如果长度一致，但index有不一致的，对应的值也会被标记为not number。

In [36]:
sr1=pd.Series([12,23,24],index=['c','a','d'])
sr2=pd.Series([11,20,10],index=['d','c','a'])
print(sr1+sr2)
sr3=pd.Series([11,20,10,40],index=['d','c','a','b'])
print(sr1+sr3)

a    33
c    32
d    35
dtype: int64
a    33.0
b     NaN
c    32.0
d    35.0
dtype: float64


- **缺失值处理**：not number其实是一种缺失值，pandas提供了处理方法。
判断是否是缺失值,可以用isnull/notnull方法，由于pd支持布尔索引，所以可以利用这个方法去掉缺失值。当然，有一个更直接的函数dropna

In [16]:
sr=sr1+sr3
print(sr.isnull())
print(sr.notnull())
print(sr[sr.notnull()])
print(sr.dropna())

a    False
b     True
c    False
d    False
dtype: bool
a     True
b    False
c     True
d     True
dtype: bool
a    33.0
c    32.0
d    35.0
dtype: float64
a    33.0
c    32.0
d    35.0
dtype: float64


- **缺失值填充**：使用fillna函数，将缺失值赋值为给定的参数。
当然，需要注意无论对缺失值怎样处理，原来的Series是不会变的，要保存的话就重新创建一个变量

In [15]:
print(sr.fillna(0))
print(sr.fillna(sr.mean()))

a    33.0
b     0.0
c    32.0
d    35.0
dtype: float64
a    33.000000
b    33.333333
c    32.000000
d    35.000000
dtype: float64


2. DataFrame


- **切片索引**：用index获取行索引，columns获取列索引，values获取值，T进行转置，describe进行数据说明。对于一个df，其本质上是多个series作为列组合在一起，所以单一切片时自然默认是把列切出来。df切片只能切单一维度，列或行，并且切行的时候只能切多行，这也很好理解，因为切单行，输入一个参数，程序不知道是行还是列，根据前面的规则，默认按列查找。loc与iloc两个方法是应对行和列同时切片的方法，前者是按标签索引，后者是按位置索引，按照标签索引是很自然的方式，因此默认索引方式是loc，这个默认意思是对于输入对象认为是标签而非位置，比如df[3]是对列名为3而非第四列进行切片。在切片时，行和列的要求用逗号隔开要求即可，如果只想切行，后面的逗号冒号可以都不加（你或许认为这个解释了前面df也可以切行，但是只能切多行的原因，觉得是loc方法，但实际上df[1:3]，虽然是切多行，而且是按位置，并没有使用iloc方法，也能切出1到2行），只想切列的话，前面的冒号逗号要加。

In [5]:

import pandas as pd
df=pd.DataFrame({'one':[1,2,3],'two':[4,5,6]},index=['a','b','c'])
print(df)
print(df['one']['a'],end='')#先列后行，但是用了两次中括号，不推荐
#以下注意区分
df['one']#切第一列
df[['one','two']]#索引两列
df[1]#输入了数字，默认按列和标签查找，实际上不存在列名为1的列，就会报错
df[1:2]#输入了位置的范围，并不会输出第一列，而是会输出第一行
df['a':'b']#输入了标签的范围，会输出第一行和第二行
df['one':'two']#仿照上面的，能不能索引列呢？答案是不行
#以上对于df的直接切片可以理解为，由于没有引进loc/iloc等方法，所以默认除了df['one'],df[['one','two']]之外，都
#将其认为是对行进行索引，以至于像df[1:2]，df['a':'b']这样对位置或标签进行索引，都能得到列。
#也可以看到其混乱之处，并且如果出现整数索引，更是容易出错，所以强烈推荐loc/iloc方法进行索引。

   one  two
a    1    4
b    2    5
c    3    6
1

- **缺失值处理**：在进行缺失值处理时，大部分情况与series是相似的，但是也有不同。比如dropna方法，参数有df.dropna(axis=,how=)

In [33]:
df.loc['a']=np.NAN
df.loc['b','one']=np.NAN
print(df)
print(df.dropna(how='all')) #all参数就表示只有这一行全是缺失值时才drop
print(df.dropna(how='any')) #any参数表示只要有缺失值就drop
print(df.dropna(axis=1)) #axis参数默认是0，以行为x轴，所以drop时我们是省略掉一行，我们将其设置为1，这样drop会省略一列

   one  two
a  NaN  NaN
b  NaN  5.0
c  3.0  6.0
   one  two
b  NaN  5.0
c  3.0  6.0
   one  two
c  3.0  6.0
Empty DataFrame
Columns: []
Index: [a, b, c]


- **常用函数**：dataframe排序有按值排序和按索引排序两种,有缺失值的部分不参与排序，放在最后面（不论是升序还是降序）

In [38]:
df.sort_values(by='two',ascending=False) #按照值并且是按列排序，降序
df.sort_values(by='c',ascending=False,axis=1) #按照值并且是按行排序，降序
df.sort_index(ascending=False,axis=1) #按照标签排序，降序，并且对列索引进行排序。由于行索引或者列索引都是只有一个，因此就不需要by参数了

,two,one
a,NaN,NaN
b,5.0,NaN
c,6.0,3.0


3. 时间序列

- **时间对象生成**：一般有两种情况，把输入的字符串转化为时间格式，或者是生成一个时间范围

In [51]:
import dateutil
pd.to_datetime(['2023-10-01','6/12/1998','2049-JAN-01','12/30/2002'],format='mixed') #将format参数设置成mixed，就可以识别各种类型的数据，转化为时间索引
pd.date_range('2023-01-01','2024-01-01',freq='D')#start开始时间，end结束时间，periods时间长度，freq时间频率，默认为D(ay)，可选H(our),W(eek),B(usiness),A(year)

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08',
               '2023-01-09', '2023-01-10',
               ...
               '2023-12-23', '2023-12-24', '2023-12-25', '2023-12-26',
               '2023-12-27', '2023-12-28', '2023-12-29', '2023-12-30',
               '2023-12-31', '2024-01-01'],
              dtype='datetime64[ns]', length=366, freq='D')

- **切片索引**：如果把时间序列对象作为series的index，切片也会变得很灵活

In [57]:
T_Series=pd.Series(np.arange(100),index=pd.date_range('2023-01-01',periods=100,freq='D'))
T_Series['2023-03'] #只选取3月份数据
T_Series.resample('M').sum() #reample按照给定参数M，对每月数据进行抽取，sum函数求和
T_Series.truncate(after='2023-3-01') #对三月份前的数据进行切片


2023-01-01     0
2023-01-02     1
2023-01-03     2
2023-01-04     3
2023-01-05     4
2023-01-06     5
2023-01-07     6
2023-01-08     7
2023-01-09     8
2023-01-10     9
2023-01-11    10
2023-01-12    11
2023-01-13    12
2023-01-14    13
2023-01-15    14
2023-01-16    15
2023-01-17    16
2023-01-18    17
2023-01-19    18
2023-01-20    19
2023-01-21    20
2023-01-22    21
2023-01-23    22
2023-01-24    23
2023-01-25    24
2023-01-26    25
2023-01-27    26
2023-01-28    27
2023-01-29    28
2023-01-30    29
2023-01-31    30
2023-02-01    31
2023-02-02    32
2023-02-03    33
2023-02-04    34
2023-02-05    35
2023-02-06    36
2023-02-07    37
2023-02-08    38
2023-02-09    39
2023-02-10    40
2023-02-11    41
2023-02-12    42
2023-02-13    43
2023-02-14    44
2023-02-15    45
2023-02-16    46
2023-02-17    47
2023-02-18    48
2023-02-19    49
2023-02-20    50
2023-02-21    51
2023-02-22    52
2023-02-23    53
2023-02-24    54
2023-02-25    55
2023-02-26    56
2023-02-27    57
2023-02-28    

4. 文件处理

- **文件读取**：主要是pd.read_csv()函数，参数包含sep：指定分隔符；header=None就是说这个文件没有列名；names可以指定列名；index_col制定某一列为索引；parse_dates，如果等于True，就会把所有可以解释为时间的值转换为时间数据，也可以等于一个列表，比如['date']，把这一列数据变为时间变量；na_values，指定哪些数据是缺失值，也是传入一个列表